In [ ]:
from bs4 import BeautifulSoup as bs
import urllib.request
import re
import pickle
import json

In [ ]:
all_albums = []

for year in range(2002,2018) :
    
    url = "https://www.billboard.com/charts/year-end/"+str(year)+"/top-billboard-200-albums"
    req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    html = urllib.request.urlopen(req)
    res = bs(html.read(),"html5lib")
    album_titles = [re.sub("\n", '', c.text) for c in res.findAll(attrs={'class': "ye-chart-item__title"})]
    album_artists = [re.sub("\n", '', c.text) for c in res.findAll(attrs={'class': "ye-chart-item__artist"})]
    album_image_src = [c.find("img")["src"] for c in res.findAll(attrs={'class': "ye-chart-item__image"})]
    
    print(len(album_titles), len(album_artists), len(album_image_src))
    
    
    all_albums.extend([{'album': x[0], 'artist' : x[1], 'year' : year, 'rank' : r + 1, 'img_src' : x[2]} for r, x in enumerate(zip(album_titles, album_artists, album_image_src))])
    
all_albums[:10]

# save songs

In [ ]:
with open('all_albums.json', 'w') as fp:
    json.dump(all_albums, fp)